In [1]:
# change current working directory to the root path
# to enable relative imports
import os
curr_path = os.getcwd()
root_path = os.path.dirname(curr_path)
os.chdir(root_path)

import dotenv
dotenv.load_dotenv()

from lib.database import CRUD


# DB_HOST=tester3.kaist.ac.kr
# DB_PORT=15432
# DB_USER=yangheechan
# DB_PASSWORD=yang1234
# DB=yangheechan
db = CRUD(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB"),
    slack_channel=os.getenv("SLACK_CHANNEL"),
    slack_token=os.getenv("SLACK_TOKEN")
)


In [2]:
# merge all bit sequences into a single string
# the index of the bit is 1 if the bit is 1, otherwise 0

def cov_bit_seq(tcs_results):
    bit_val = 0
    for item in tcs_results:
        # print(item)
        bit_seq_str = item[0]
        bit_seq = int(bit_seq_str, 2)  # Convert binary string to integer
        bit_val |= bit_seq  # Use bitwise OR to merge sequences
    return format(bit_val, f'0{len(tcs_results[0][0])}b')  # Convert back to binary string

PID = "Time"
FID = 243
BID = 13

# PASSING TEST CASES
passing_tcs = db.read(
    "d4j_tc_info",
    "line_coverage_bit_sequence",
    conditions = {
        "fault_idx": FID, # This is specific to pid:bid
        "result": 0,
    }
)
print(f"Number of passing test cases: {len(passing_tcs)}")

bit_length = len(passing_tcs[0][0])

passing_tc_covered_bit_seq_str = cov_bit_seq(passing_tcs)

# FAILING TEST CASES
failing_tcs = db.read(
    "d4j_tc_info",
    "line_coverage_bit_sequence",
    conditions = {
        "fault_idx": FID,
        "result": 1,
    }
)
print(f"Number of failing test cases: {len(failing_tcs)}")

failing_tc_covered_bit_seq_str = cov_bit_seq(failing_tcs)

# Lets find the unique lines covered by failing tcs
bit_sequences = []
# failing_tc_covered_bit_seq is the lines covered by failing TCS (vise versa for passing TCS)
# What is the bitwise operation to find the unique lines covered by failing TCS? 
unique_failing_lines = int(failing_tc_covered_bit_seq_str, 2) & ~int(passing_tc_covered_bit_seq_str, 2)
unique_failing_lines_str = format(unique_failing_lines, f'0{bit_length}b')

ground_truth_line_idx = []
for i, bit in enumerate(unique_failing_lines_str):
    if bit == "1":
        print(f"Bit {i} is set to 1")
        ground_truth_line_idx.append(i)

Number of passing test cases: 369
Number of failing test cases: 2
Bit 7 is set to 1
Bit 33 is set to 1
Bit 34 is set to 1
Bit 35 is set to 1
Bit 36 is set to 1
Bit 37 is set to 1
Bit 38 is set to 1
Bit 39 is set to 1
Bit 40 is set to 1
Bit 41 is set to 1
Bit 42 is set to 1
Bit 43 is set to 1
Bit 44 is set to 1
Bit 45 is set to 1
Bit 46 is set to 1
Bit 47 is set to 1
Bit 48 is set to 1
Bit 49 is set to 1
Bit 50 is set to 1
Bit 51 is set to 1
Bit 52 is set to 1
Bit 53 is set to 1
Bit 54 is set to 1
Bit 64 is set to 1
Bit 65 is set to 1
Bit 66 is set to 1
Bit 67 is set to 1
Bit 68 is set to 1
Bit 69 is set to 1
Bit 200 is set to 1
Bit 201 is set to 1
Bit 202 is set to 1
Bit 219 is set to 1
Bit 244 is set to 1
Bit 245 is set to 1
Bit 246 is set to 1
Bit 261 is set to 1
Bit 262 is set to 1
Bit 263 is set to 1
Bit 264 is set to 1
Bit 265 is set to 1
Bit 266 is set to 1
Bit 267 is set to 1
Bit 268 is set to 1
Bit 269 is set to 1
Bit 270 is set to 1
Bit 271 is set to 1
Bit 272 is set to 1
Bit 

In [3]:
line_infos = db.read(
    "d4j_line_info",
    "line_idx, file, method, line_num",
    conditions={
        "fault_idx": FID,
    }
)

ground_truth_data = {}

# select 8 randomly from ground_truth_line_idx
import random
random.shuffle(ground_truth_line_idx)
ground_truth_line_idx = ground_truth_line_idx[:8]
for gt in ground_truth_line_idx:
    print(f"Ground truth line index: {gt}")

for line_info in line_infos:
    line_idx = line_info[0]
    file = line_info[1]
    method = line_info[2]
    line_num = line_info[3]

    if line_idx in ground_truth_line_idx:
        if file not in ground_truth_data:
            ground_truth_data[file] = {}
        if method not in ground_truth_data[file]:
            ground_truth_data[file][method] = []
        ground_truth_data[file][method].append((line_num, line_idx))

import json
print(json.dumps(ground_truth_data, indent=2))

Ground truth line index: 403
Ground truth line index: 423
Ground truth line index: 262
Ground truth line index: 449
Ground truth line index: 274
Ground truth line index: 268
Ground truth line index: 486
Ground truth line index: 272
{
  "org/joda/time/format/DateTimeFormat.java": {
    "<clinit>()": [
      [
        149,
        262
      ]
    ],
    "parsePatternTo(org.joda.time.format.DateTimeFormatterBuilder,java.lang.String)": [
      [
        405,
        268
      ],
      [
        412,
        272
      ],
      [
        479,
        274
      ]
    ]
  },
  "org/joda/time/format/DateTimeFormatterBuilder.java": {
    "appendLiteral(java.lang.String)": [
      [
        408,
        403
      ]
    ],
    "appendYear(int,int)": [
      [
        829,
        423
      ]
    ]
  },
  "org/joda/time/format/ISODateTimeFormat.java": {
    "weekElement()": [
      [
        1623,
        449
      ]
    ],
    "dateElementParser()": [
      [
        634,
        486
      ]
    ]

In [4]:
gid = 0

for file, methods in ground_truth_data.items():
    print(f"File: {file}")
    for method, line_info in methods.items():
        for line_num, line_idx in line_info:
            # Assuming the description is "FAULT_OF_OMISSION"
            # You can change this to whatever description you want
            gid += 1
            value = [PID, BID, gid, file, method, line_num, line_idx, "FAULT_OF_OMISSION"]
            print(value)
            db.insert(
                "d4j_ground_truth_info",
                "pid, bid, gid, file, method, line, line_idx, description",
                value
            )

File: org/joda/time/format/DateTimeFormat.java
['Time', 13, 1, 'org/joda/time/format/DateTimeFormat.java', '<clinit>()', 149, 262, 'FAULT_OF_OMISSION']
['Time', 13, 2, 'org/joda/time/format/DateTimeFormat.java', 'parsePatternTo(org.joda.time.format.DateTimeFormatterBuilder,java.lang.String)', 405, 268, 'FAULT_OF_OMISSION']
['Time', 13, 3, 'org/joda/time/format/DateTimeFormat.java', 'parsePatternTo(org.joda.time.format.DateTimeFormatterBuilder,java.lang.String)', 412, 272, 'FAULT_OF_OMISSION']
['Time', 13, 4, 'org/joda/time/format/DateTimeFormat.java', 'parsePatternTo(org.joda.time.format.DateTimeFormatterBuilder,java.lang.String)', 479, 274, 'FAULT_OF_OMISSION']
File: org/joda/time/format/DateTimeFormatterBuilder.java
['Time', 13, 5, 'org/joda/time/format/DateTimeFormatterBuilder.java', 'appendLiteral(java.lang.String)', 408, 403, 'FAULT_OF_OMISSION']
['Time', 13, 6, 'org/joda/time/format/DateTimeFormatterBuilder.java', 'appendYear(int,int)', 829, 423, 'FAULT_OF_OMISSION']
File: org/jo